In [ ]:
import random
import namegenerator
import hashlib as hasher
import datetime as date
import torch
import syft

In [ ]:
hook = syft.TorchHook(torch)
names = list()
transaction = list()
labels = list()

In [ ]:
i = 0
while i < 50:
    name = namegenerator.gen()
    names.append(name)
    transaction_id = random.randint(100000000000000,999999999999999)
    transaction.append(transaction_id)
    label = random.randint(0,1)
    labels.append(label)
    i = i + 1

In [ ]:
for i in range(20):
    print( "Name -->",names[i] , "Transaction id -->:",transaction[i], "Label -->",labels[i])

In [ ]:
class Block:    
    def __init__(self, name, transaction_id,label):
        self.name = name
        self.transaction_id = transaction_id
        self.label = label
        self.hash = self.hash_block()

    def hash_block(self):
        sha = hasher.sha256()
        sha.update(str(self.name).encode('utf-8') + 
                   str(self.transaction_id).encode('utf-8') + 
                   str(self.label).encode('utf-8'))
        return sha.hexdigest()

In [ ]:
name_0 = names[0]
transaction_0 = transaction[0]
labels_0 = labels[0]
name_0,transaction_0,labels_0

In [ ]:
def create_genesis_block():
    
    return Block(name_0, transaction_0,labels_0)

In [ ]:
def next_block(last_block,j):
    this_name = names[j]
    this_transaction_id = transaction[j]
    this_label = labels[j]
    this_hash = last_block.hash
    return Block(this_name, this_transaction_id,label)

In [ ]:
blockchain = [create_genesis_block()]
previous_block = blockchain[0]
num_of_blocks_to_add = len(names)


In [ ]:
for i in range(0, num_of_blocks_to_add):
    block_to_add = next_block(previous_block,i)
    blockchain.append(block_to_add)
    previous_block = block_to_add
    print("Name: {}\n".format(block_to_add.name))
    print("transaction_id: {}\n".format(block_to_add.transaction_id))
    print("Label: {}\n".format(block_to_add.label))    
    print("Hash: {}\n".format(block_to_add.hash))

In [ ]:
a = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z","aa","bb","cc",
     "dd","ee","ff","gg","hh","ii","jj","kk","ll","mm","nn","oo","pp","qq","rr","ss","tt","uu","vv","ww","xx"]

In [ ]:
b = a

In [ ]:
for i in range(len(names)):
    names[i] = syft.VirtualWorker(hook, id = names[i])
    a[i] = torch.tensor([transaction[i]]).send(names[i])
    b[i] = torch.tensor([labels[i]]).send(names[i])

In [ ]:
for i in range(len(a)):
    print("Transaction_id address -->", a[i],"\n Label address -->",b[i])

In [ ]:
datasets = []

In [ ]:
for i in range(len(names)):
    datasets.append((a[i],b[i]))

In [ ]:
for i in range(10):
    print(datasets[i])

In [ ]:
from torch import nn
from torch import optim

In [ ]:
def train(iterations = 20):
    model = nn.Linear(50,22)
    optimizer_fed = optim.SGD(params = model.parameters(), lr = 0.1)
    for iter in range(iterations):
        for data, target  in datasets:
            model = model.send(data.location)
            optimizer_fed.zero_grad()
            pred = model(data)
            loss = (( pred - target) ** 2).sum()
            loss.backward()
            optimizer_fed.step()
            model = model.get()
            print(loss.get())


In [ ]:
train()